<a href="https://colab.research.google.com/github/deniskapel/GenerativeChitChat/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget "https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt"
pip install -r requirements.txt

In [2]:
import codecs
import json
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [ ]:
%%bash
mkdir data
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE" -O data/qa_data.jsonl.zip && rm -rf /tmp/cookies.txt
unzip data/qa_data.jsonl.zip -d data

## Data

In [4]:
!sed 5q data/qa_data.jsonl

{"question": "долго ли идут деньги с яндексденег на карту visa?", "category": "Бизнес, Финансы", "responses": ["нет. прорыв 35 ;)"]}
{"question": "можно ли зарегистрировать авто в другом регионе", "category": "Авто, Мото", "responses": ["можно на родственника из того региона.. .  а потом ездить по доверке"]}
{"question": "что делать если у меня очень тонкие ногти а хочется их отрастить?", "category": "Красота и Здоровье", "responses": ["витамины и умная эмаль (каждый день)", "ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.", "умная эмаль, витамины, йод, и поменьше крась лаком ", "лаки фирмы trind производство usa + кальций"]}
{"question": "в чем отличие медитации от йоги?", "category": "Спорт", "responses": ["букв в йоге меньше", "в медитации ты просто сидишь и мммммычишь. а в йоге всяко разные упражнения вытворяешь", "в медитации вроде просто тупо сидишь и успокаеваешься, а в йоге еще и ноги за уши закидывать надо"]}
{"question": "когда нач

In [4]:
with codecs.open("data/qa_data.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))

data = []
for line in tqdm(lines):
    for response in line['responses']:
        data.append(
            {'question': line['question'].lower(),
             'category': line['category'],
             'response': response.lower()})

del lines
df = pd.json_normalize(data)
del data

  0%|          | 0/2808811 [00:00<?, ?it/s]

In [5]:
df.head()

,question,category,response
0,долго ли идут деньги с яндексденег на карту visa?,"Бизнес, Финансы",нет. прорыв 35 ;)
1,можно ли зарегистрировать авто в другом регионе,"Авто, Мото",можно на родственника из того региона.. . а п...
2,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,витамины и умная эмаль (каждый день)
3,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,ванночки с морской солью. с вечера мажь ногти ...
4,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,"умная эмаль, витамины, йод, и поменьше крась л..."


In [28]:
df_mini = df.sample(frac=0.01, random_state=42)
train_df=df_mini.sample(frac=0.8,random_state=42)
test_df=df_mini.drop(train_df.index)

In [29]:
train_df.category.value_counts()

Знакомства, Любовь, Отношения    7978
Философия, Непознанное           7098
Семья, Дом, Дети                 6282
Красота и Здоровье               5233
Искусство и Культура             5192
Авто, Мото                       4711
Спорт                            3984
Животные, Растения               3808
Досуг, Развлечения               3320
Еда, Кулинария                   2804
Стиль, Мода, Звезды              2744
Бизнес, Финансы                  2366
Путешествия, Туризм              2298
Работа, Карьера                  2287
Образование                      2032
Name: category, dtype: int64

In [30]:
tokenizer = yttm.BPE(model="pretrained/pretrained_bpe_lm.model")

In [31]:
train_df.question = tokenizer.encode(train_df.question.tolist(), bos=True, eos=True)
train_df.response = tokenizer.encode(train_df.response.tolist(), bos=True, eos=True)
test_df.question = tokenizer.encode(test_df.question.tolist(), bos=True, eos=True)
test_df.response = tokenizer.encode(test_df.response.tolist(), bos=True, eos=True)

In [39]:
q_lengths = np.array([len(x) for x in train_df.question.tolist()])
r_lenghts = np.array([len(x) for x in train_df.response.tolist()])
np.percentile(q_lengths, q=95), np.percentile(r_lenghts, q=95)

(28.0, 29.0)

In [40]:
batch_size = 64
max_len = 32
pad_index = 0
eos_index = 3